1.0 Import and install python libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

It's very important that you set this! In this course we will use the random seed value of 1.

In [2]:
# set random seed to ensure that results are repeatable
np.random.seed(1)

2.0 Load and clean the data

In [3]:
# load data
universalBank = pd.read_csv(r"C:\DSP\WE04\UniversalBank.csv")

3.0 Conduct initial exploration of the data

In [4]:
# look at the data
universalBank.head(3) # note that we don't want to dump all the data to the screen

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0


In [5]:
# generate a basic summary of the data
universalBank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  5000 non-null   int64  
 1   Age                 5000 non-null   int64  
 2   Experience          5000 non-null   int64  
 3   Income              5000 non-null   int64  
 4   ZIP Code            5000 non-null   int64  
 5   Family              5000 non-null   int64  
 6   CCAvg               5000 non-null   float64
 7   Education           5000 non-null   int64  
 8   Mortgage            5000 non-null   int64  
 9   Personal Loan       5000 non-null   int64  
 10  Securities Account  5000 non-null   int64  
 11  CD Account          5000 non-null   int64  
 12  Online              5000 non-null   int64  
 13  CreditCard          5000 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 547.0 KB


In [6]:
# generate a statistical summary of the numeric value in the data
universalBank.describe()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.937938,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,1.747659,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,0.700000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,1.500000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,2.500000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,10.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [7]:
# Check the missing values by summing the total na's for each variable
universalBank.isna().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [8]:
# create a list of these catagorical variables
category_var_list = list(universalBank.select_dtypes(include='object').columns)
category_var_list

[]

In [9]:
# explore the categorical variable values - often there are typos here that need to be fixed.
for cat in category_var_list: # generally, we want to avoid for loops and use a functional style (i.e. list comprehension)
    print(f"Category: {cat} Values: {universalBank[cat].unique()}")

Process the data

In [10]:
dummies_df = pd.get_dummies(universalBank['ID'], prefix='ID', drop_first=True)
universalBank = universalBank.join(dummies_df)
universalBank.drop('ID', axis=1, inplace = True)

In [11]:
universalBank = universalBank.join(pd.get_dummies(universalBank['ZIP Code'], prefix='ZIP Code', drop_first=True))
universalBank.drop('ZIP Code', axis=1, inplace = True)

In [12]:
# explore the dataframe columns to verify dropped columns
universalBank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 5477 entries, Age to ZIP Code_96651
dtypes: float64(1), int64(11), uint8(5465)
memory usage: 26.5 MB


3.0 Split data (train/test)

In [13]:
# split the data into validation and training set
train_df, test_df = train_test_split(universalBank, test_size=0.3)

# to reduce repetition in later code, create variables to represent the columns
# that are our predictors and target
target = 'Securities Account'
predictors = list(universalBank.columns)
predictors.remove(target)

Standardize numeric values
Now, let's create a common scale between the numberic columns by standardizing each numeric column

In [14]:
# create a standard scaler and fit it to the training set of predictors
scaler = preprocessing.StandardScaler()
cols_to_stdize = predictors                
               
# Transform the predictors of training and validation sets
train_df[cols_to_stdize] = scaler.fit_transform(train_df[cols_to_stdize]) # train_predictors is not a numpy array


test_df[cols_to_stdize] = scaler.transform(test_df[cols_to_stdize]) # validation_target is now a series object

In [15]:
#Save the data
train_X = train_df[predictors]
train_y = train_df[target] # train_target is now a series objecttrain_df.to_csv('airbnb_train_df.csv', index=False)
test_X = train_df[predictors]
test_y = test_df[target] # validation_target is now a series object

train_df.to_csv(r'C:\DSP\WE04\universalBank_train_df.csv', index=False)
train_X.to_csv(r'C:\DSP\WE04\universalBank_train_X.csv', index=False)
train_y.to_csv(r'C:\DSP\WE04\universalBank_train_y.csv', index=False)
test_df.to_csv(r'C:\DSP\WE04\universalBank_test_df.csv', index=False)
test_X.to_csv(r'C:\DSP\WE04\universalBank_test_X.csv', index=False)
test_y.to_csv(r'C:\DSP\WE04\universalBank_test_y.csv', index=False)

In [16]:
#Load the saved data
X_train = pd.read_csv(r'C:\DSP\WE04\universalBank_train_X.csv') 
y_train = pd.read_csv(r'C:\DSP\WE04\universalBank_train_y.csv') 
X_test = pd.read_csv(r'C:\DSP\WE04\universalBank_test_X.csv') 
y_test = pd.read_csv(r'C:\DSP\WE04\universalBank_test_y.csv') 

4.0 Logistic Regression using RandomSearch

In [ ]:
score_measure = "precision"
kfolds = 5

param_grid = {'C': np.arange(5,15), # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(300,700)
    
    
}

logreg_rs = LogisticRegression()
rand_search = RandomizedSearchCV(estimator = logreg_rs, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestlr = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


5.0 Logistic Regression using GridSearch

In [ ]:
score_measure = "precision"
kfolds = 5

param_grid = {'C': np.arange(7,10), # C is the regulization strength
               'penalty':['l1', 'l2','elasticnet','none'],
              'solver':['saga','liblinear'],
              'max_iter': np.arange(400,700)
    
    
}
logreg_gs =  LogisticRegression()
grid_search = GridSearchCV(estimator = logreg_gs, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestlogreg = grid_search.best_estimator_

6.0 Polynomial kernal SVM classification model using random search

In [27]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'C': np.arange(5,15),
    'degree': [3, 4,5],
    'gamma': ['scale', 'auto'],
    'kernel':['poly']
    
}

Poly_kernal_R_SVM = SVC()
rand_search = RandomizedSearchCV(estimator = Poly_kernal_R_SVM, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 60 candidates, totalling 300 fits


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 60 is smaller than n_iter=500. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best precision score is 0.6889031339031338
... with parameters: {'kernel': 'poly', 'gamma': 'auto', 'degree': 3, 'C': 6}


7.0 Polynomial kernal SVM classification model using grid search

In [33]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'C': np.arange(3,10),
    'degree': [2, 3,4],
    'gamma': ['scale', 'auto'],
    'kernel':['poly']
    
}

Poly_kernal_G_SVM = SVC()
grid_search = GridSearchCV(estimator = Poly_kernal_G_SVM, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 42 candidates, totalling 210 fits


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best precision score is 0.6905372405372405
... with parameters: {'C': 3, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}


8.0 Linear kernal SVM classification model using random search

In [32]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'C': np.arange(5,30),
    'degree': [3, 4,5],
    'gamma': ['scale', 'auto'],
    'kernel':['linear']
    
}

linear_kernal_R_SVM = SVC()
rand_search = RandomizedSearchCV(estimator = linear_kernal_R_SVM, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 150 candidates, totalling 750 fits


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 150 is smaller than n_iter=500. Running 150 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best precision score is 0.6249743589743589
... with parameters: {'kernel': 'linear', 'gamma': 'scale', 'degree': 3, 'C': 5}


9.0 Linear kernal SVM classification model using Grid search

In [36]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'C': np.arange(2,30),
    'degree': [3, 4,5],
    'gamma': ['scale', 'auto'],
    'kernel':['linear']
    
}

linear_kernal_G_SVM = SVC()
grid_search = GridSearchCV(estimator = linear_kernal_G_SVM, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 168 candidates, totalling 840 fits


C:\Users\Sravani\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best precision score is 0.6249743589743589
... with parameters: {'C': 2, 'degree': 3, 'gamma': 'scale', 'kernel': 'linear'}


10. rbf kernal SVM classification model using random search

In [ ]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'C': np.arange(5,15),
    'degree': [3, 4,5],
    'gamma': ['scale', 'auto'],
    'kernel':['rbf']
    
}

rbf_kernal_R_SVM = SVC()
rand_search = RandomizedSearchCV(estimator = rbf_kernal_R_SVM, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

11.rbf kernal SVM classification model using Grid search

In [ ]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'C': np.arange(8,17),
    'degree': [2, 3,4],
    'gamma': ['scale', 'auto'],
    'kernel':['poly']
    
}

rbf_kernal_G_SVM = SVC()
grid_search = GridSearchCV(estimator = rbf_kernal_G_SVM, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

12.Conduct an exhaustive search across a smaller range of parameters around the parameters found in the initial random search.

In [ ]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,53),  
    'min_samples_leaf': np.arange(1,53),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 53), 
    'max_depth': np.arange(1,53), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

13.Conduct an exhaustive search across a smaller range of parameters around the parameters found in the initial random search. 

In [ ]:
score_measure = "precision"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(5,9),  
    'min_samples_leaf': np.arange(6,10),
    'min_impurity_decrease': np.arange(0.0048, 0.0054, 0.0001),
    'max_leaf_nodes': np.arange(5,10), 
    'max_depth': np.arange(1,10), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

As per the results among the above models according to precision score polynomial kernal model with grid search stands out best with 0.6905372405372405 score and second best goes to Polynomial kernal SVM classification model using random search with 0.6889031339031338 score. So finally polynomial kernal model with grid search fits best followed by Polynomial kernal SVM classification model using random searchwhen compared to others.